In [ ]:
from hydra import compose, initialize
from omegaconf import OmegaConf

initialize(config_path="config")


In [ ]:

import gym
import json
from pathlib import Path
import wandb
import hydra
from omegaconf import DictConfig, OmegaConf
import logging
import os.path
import sys
import cv2

from gym.wrappers.monitoring.video_recorder import ImageEncoder
from stable_baselines3.common.vec_env.base_vec_env import tile_images

import carla
from carla_gym.utils import config_utils
from utils import server_utils
from mile.agents.rl_birdview.utils.wandb_callback import WandbCallback
from mile.constants import CARLA_FPS
import time
from mile.agents.mile.mile_agent import MileAgent


obs_configs = {
    'hero': {'speed': {'module': 'actor_state.speed'}, 'gnss': {'module': 'navigation.gnss'},
             'central_rgb': {'module': 'camera.rgb', 'fov': 100, 'width': 960, 'height': 600,
                             'location': [-1.5, 0.0, 2.0], 'rotation': [0.0, 0.0, 0.0]},
             'route_plan': {'module': 'navigation.waypoint_plan', 'steps': 20},
             'birdview': {'module': 'birdview.chauffeurnet', 'width_in_pixels': 192,
                          'pixels_ev_to_bottom': 32, 'pixels_per_meter': 5.0,
                          'history_idx': [-16, -11, -6, -1], 'scale_bbox': True, 'scale_mask_col': 1.0}}
}

reward_configs = {'hero': {'entry_point': 'reward.valeo_action:ValeoAction'}}
terminal_configs = {'hero': {'entry_point': 'terminal.leaderboard:Leaderboard'}}
env_configs = {'carla_map': 'Town02', 'routes_group': None, 'weather_group': 'new'}


def main(cfg: DictConfig):
    agents_dict = {}
    for ev_id, ev_cfg in cfg.actors.items():
        cfg_agent = cfg.agent[ev_cfg.agent]
        OmegaConf.save(config=cfg_agent, f='config_agent.yaml')
        agents_dict[ev_id] = MileAgent('config_agent.yaml')

    env_idx = 0

    ckpt_task_idx = 0
    # compose suite_name
    env_setup = OmegaConf.to_container(cfg.test_suites[env_idx])
    suite_name = '-hero' + '_' + env_setup['env_id']
    for k in sorted(env_setup['env_configs']):
        suite_name = suite_name + '_' + str(env_setup['env_configs'][k])

    agents_log_dir = Path('agents_log') / suite_name
    agents_log_dir.mkdir(parents=True, exist_ok=True)
    
    server_utils.kill_carla(cfg.port)
    server_manager = server_utils.CarlaServerManager(cfg.carla_sh_path, port=cfg.port)
    server_manager.start()

    env = gym.make(env_setup['env_id'], obs_configs=obs_configs, reward_configs=reward_configs,
                   terminal_configs=terminal_configs, host=cfg.host, port=cfg.port,
                   seed=cfg.seed, no_rendering=cfg.no_rendering, **env_configs)

    task_idx = 0
    env.set_task_idx(task_idx)
    run_name = f"{env.task['weather']}_{env.task['route_id']:02d}"

    agent = agents_dict['hero']
    actor_id = 'hero'
    log_dir = agents_log_dir / actor_id
    log_dir.mkdir(parents=True, exist_ok=True)
    agent.reset(log_dir / f'{run_name}.log')

    obs = env.reset()
    timestamp = env.timestamp
    done = {'__all__': False}

    for counter in range(1000):
        print(counter)
        control_dict = {}
        control_dict[actor_id] = carla.VehicleControl(
            throttle=0.843839, steer=0.035816, brake=0.000000)
        agent.run_step(obs[actor_id], timestamp)

        obs, reward, done, info = env.step(control_dict)

        img = agent.render(info[actor_id]['reward_debug'], info[actor_id]['terminal_debug'])
        cv2.imshow('a', img)
        cv2.waitKey(1)

    env.close()
    server_manager.stop()

    
cfg = compose(config_name="evaluate", overrides=[
    "carla_sh_path=/home/carla/CarlaUE4.sh", "agent.mile.ckpt=/home/carla/mile.ckpt", "port=2000"])
main(cfg)
